In [1]:
import numpy as np
import pandas as pd

In [2]:
%load_ext autoreload
%autoreload 2

https://openaccess.uoc.edu/bitstream/10609/148363/3/miravecaTFG0623memoria.pdf

https://www.portalinmobiliario.com/robots.txt


In [3]:
from webscrapper_portal_inmobiliario import *

### TODO:
- ~~trackear propiedades en el tiempo para el mismo sector  SE HACE CON LA [LAT LONG]~~
- Sortear baneos de ip por muchas request - sectores con muchas propiedades
https://brightdata.com/cp/zones/residential_proxy1/edit 8 dolares el GB
- Calculos de estadicticas de rentabilidad
- rankear propiedades basado en esto
- grafico mapa donde se encuentran estas propiedades
- extraer imagenes del TOP 3 Propiedades

### FIXES
- ~~Cambiar id unico al titulo, se pierden precios con lat long porque poseen la misma ubicacion~~
- ~~hacer que cuando falle, re llenenen con nan los valores faltantes~~
- ~~al cambiar en total number of request max--> 0, se pierde el hilo para sacar la tabla fallando~~
- ~~las tablas no estan siendo detectadas por xpath ni en soup tables.. aparentemente quedan en un formato lazy,
   pero la data esta ahi en un diccionario~~
   - ~~El problema sucedia porque se requere un wait para que carge todo... esto afecta aveces tambien a las paginas main urls.. se nesesita aplicar~~
- El nuevo formato de pagina low/webscrapping tiene todo lo nesesario y la full cambia en nombre al texto de "UBICACION"


0### APRENDIZAJES

webdriver
2. es bueno reiniciar y cerrar el driver
2. el estado de la pagina (restricciones) cambiara solo con el refresh!!
3. Si se aplica un init y luego get/request no retornara NADA (PUEDE QUE SI PERO EN GENERAL NO PORQUE ES UN USER NUEVO SE ASOCIARA CON WEBSCRAPPING)
4. el orden correcto sera

close--> init--> request (primera vez ,  alta probabilida de falla) -->  refresh (recarga y se puede obtener la tabla y ubicaciion)

In [4]:
WSPI = WebScraperPortalInmobiliario(tipo_operacion="inversion",
                                    tipo_inmueble="departamento",
                                    theme="default",
                                    folder_save_name="Depas_general_arriagada",
                                    debug=False)

UPDATING data for location found in Depas_general_arriagada:   0%|          | 0/100 [00:00<?, ?it/s]

#### IP PROXY

In [12]:
headers = {
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.5",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
}
proxy = WSPI.proxy_list[1]
# proxy = "35.185.196.38:3128"
r = requests.get("https://www.portalinmobiliario.com/robots.txt", headers=headers, proxies={"http": proxy, "https": proxy}, timeout=5)

if r.status_code == 200:
    print("VALID PROXY")

VALID PROXY


#### benchamarks

sleep time 2 secs-->
70 propiedades : 6 minutos
250 propiedades: 24 minutos

## sobre las 500 propiedades sufre baneos

**Rendimiento de 640 propiedades por hora -  5,6 secs por pagina**

sleep time 1 secs--> BLOCKED
**Rendimiento de x propiedades por hora -  4.8 secs por pagina**

# DATA MANAGE
------------------------------------------

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from database import *

In [13]:
conn = create_conect_db("real_state.db")
df = get_joined_data_as_dataframe(conn=conn, threshold_date="2024-07-24 15:00")
df = df.T.drop_duplicates().T
df

,Latitude,Longitude,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,...,gastos_comunes,titulo,ubicacion,link,geo_ref_name,listed,Price,Price_UF,tipo_operacion,Date
0,-33.557829,-70.58626,18.0,3.0,1.0,70.0,70.0,0.0,0.0,NaN,...,NaN,Ubicación Privilegiada. Gran Conectividad.,"Vicuña Mackenna 11847, Los Quillayes, La Flori...",https://www.portalinmobiliario.com/MLC-2628826...,Depas_general_arriagada,1,400000.0,10.628207,arriendo,2024-08-17 13:30:36.013506
1,-33.558613,-70.585941,15.0,1.0,1.0,40.12,40.12,1.0,1.0,NaN,...,67000.0,Arrienda En Un Edificio Nuevo A Pasos Del Metro,"Av. Vicuña Mackenna 10811, Los Quillayes, La F...",https://www.portalinmobiliario.com/MLC-1510438...,Depas_general_arriagada,1,370000.0,9.831091,arriendo,2024-08-17 13:30:36.022481
2,-33.557953,-70.586883,12.0,3.0,1.0,70.0,70.0,0.0,0.0,30.0,...,40000.0,Amplio Departamento En Excelente Ubicacion/ 1º...,"La Paloma 11841, Los Quillayes, La Florida, RM...",https://www.portalinmobiliario.com/MLC-2640451...,Depas_general_arriagada,1,450000.0,11.956733,arriendo,2024-08-17 13:30:36.029992
3,-33.557547,-70.586747,11.0,3.0,1.0,80.0,80.0,1.0,0.0,0.0,...,30000.0,Se Arrienda Departamento En La Florida $520.000,"La Paloma, Los Quillayes, La Florida, RM (Metr...",https://www.portalinmobiliario.com/MLC-1511448...,Depas_general_arriagada,1,520000.0,13.816669,arriendo,2024-08-17 13:30:36.036729
4,-33.559324,-70.586055,9.0,3.0,2.0,65.0,61.0,0.0,0.0,16.0,...,70000.0,Amplio Departamento (23107),"Amplio Departamento, Los Quillayes, La Florida...",https://www.portalinmobiliario.com/MLC-1512249...,Depas_general_arriagada,1,90000000.0,2391.346514,venta,2024-08-17 13:31:05.976563
5,-33.557235,-70.585874,36.0,2.0,1.0,62.0,62.0,1.0,1.0,25.0,...,90000.0,Departamento En Venta De 2 Dorm. En La Florida,"Vicuña Mackenna 6640, La Florida, La Florida, ...",https://www.portalinmobiliario.com/MLC-1503001...,Depas_general_arriagada,1,127000000.0,3374.455637,venta,2024-08-17 13:31:05.983963
6,-33.558409,-70.587695,150.0,3.0,1.0,60.0,60.0,1.0,0.0,0.0,...,30000.0,Departamento En Venta En La Florida,"El Colibrí, La Florida, Chile, Los Quillayes, ...",https://www.portalinmobiliario.com/MLC-2313468...,Depas_general_arriagada,1,109143530.0,2900.0,venta,2024-08-17 13:31:05.992315
7,-33.559993,-70.585957,300.0,3.0,1.0,56.0,56.0,0.0,0.0,17.0,...,NaN,Metro San José De La Estrella / Vicuña Mackenna,"Los Quillayes, La Florida, RM (Metropolitana)",https://www.portalinmobiliario.com/MLC-1473135...,Depas_general_arriagada,1,107000000.0,2843.0453,venta,2024-08-17 13:31:06.000168


# DATA PROCESING
------------------------------------------

In [103]:
# df_arriendo = pd.read_excel(r"C:\Users\blavin\Documents\Github_myself\portal_inmobiliario_web_scraper\results\metro_chile_españa\results_invesment_arriendo_departamento_1143_18_07_24.xlsx")

df_arriendo = pd.read_excel(r"C:\Users\blavin\Documents\Github_myself\portal_inmobiliario_web_scraper\results\Depas_general_arriagada\results_invesment_arriendo_departamento_1603_17_07_24.xlsx")

### precio arriendo segun tipologia

In [104]:
df_arriendo.groupby(by="n_dormitorios").mean()

,Unnamed: 0,latitud,longitud,precio,precio_UF,dias_desde_publicacion,n_banos,superficie_total,superficie_util,estacionamientos,bodegas,antiguedad,cantidad_pisos_edificio,piso_unidad
n_dormitorios,,,,,,,,,,,,,,
3,0.5,-33.55775,-70.586815,440000.0,11.703404,35.0,1.0,70.0,70.0,0.5,0.0,30.0,4.0,1.5


In [105]:
def remove_outliers(self, column_name):
    """ removedor de outliers para columna especificada"""
    Q1 = self[column_name].quantile(0.25)
    Q3 = self[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    condition = (self[column_name] >= lower_bound) & (self[column_name] <= upper_bound)
    return self.loc[condition]

# Attach the function to the pandas DataFrame class
pd.DataFrame.remove_outliers = remove_outliers

In [106]:
# EXTRACTOR DE PRECIO DE ARRIENDO SEGUN TIPOLOGIA
results_precio_arriendo = {}
for n_dorm in pd.Series(df_arriendo["n_dormitorios"].unique()).sort_values().values:
    clean_tipology_df = df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('precio_UF')
    precio_arriendo = int(clean_tipology_df.quantile(0.1).precio)

    results_precio_arriendo[n_dorm] = precio_arriendo
    print(f"con {n_dorm} dormitorios, se puede arrendar en aproximadamente {precio_arriendo}, analizado desde {len(clean_tipology_df)} propiedades")
    # list_results.append(df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('precio_UF'))
    
# df_price_wo_outliers = pd.concat(list_results)
# df_price_wo_outliers

con 3 dormitorios, se puede arrendar en aproximadamente 432000, analizado desde 2 propiedades


In [107]:
results_precio_arriendo

{3: 432000}

### precio por metro cuadrado

In [108]:
# EXTRACTOR DE PRECIO DE ARRIENDO SEGUN TIPOLOGIA
list_results = []
for n_dorm in pd.Series(df_arriendo["n_dormitorios"].unique()).sort_values().values:
    clean_tipology_df = df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('superficie_util')
    list_results.append(clean_tipology_df)

    # print(f"con {n_dorm} dormitorios, se puede arrendar en aproximadamente {precio_arriendo}, analizado desde {len(clean_tipology_df)} propiedades")
    # list_results.append(df_arriendo.query(f"n_dormitorios=={n_dorm}").remove_outliers('precio_UF'))

df_price_wo_outliers = pd.concat(list_results)
df_price_wo_outliers["precio_m2"] = df_price_wo_outliers.precio_UF/df_price_wo_outliers.superficie_util
promedio_zona = int(np.mean(df_price_wo_outliers.precio/df_price_wo_outliers.superficie_util))
promedio_zona_UF = np.round(np.mean(df_price_wo_outliers.precio_UF/df_price_wo_outliers.superficie_util),2)

print(f"precio/m2 de la zona es {promedio_zona} CLP , {promedio_zona_UF} UF")

precio/m2 de la zona es 6285 CLP , 0.17 UF


In [109]:
df_price_wo_outliers

,Unnamed: 0,latitud,longitud,precio,precio_UF,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,...,bodegas,antiguedad,cantidad_pisos_edificio,piso_unidad,tipo_inmueble,orientacion,titulo,ubicacion,link,precio_m2
0,0,-33.557953,-70.586883,450000,11.969390,28,3,1,70,70,...,0,30.0,4.0,1,Departamento,O,Amplio Departamento En Excelente Ubicación,"La Paloma 11841, La Florida, Chile, Los Quilla...",https://www.portalinmobiliario.com/MLC-2508405...,0.170991
1,1,-33.557547,-70.586747,430000,11.437417,42,3,1,70,70,...,0,NaN,NaN,2,Departamento,NP,"Depto Calle La Paloma N° 11.841, Comuna La Fl...","La Paloma, La Florida, Chile, Los Quillayes, L...",https://www.portalinmobiliario.com/MLC-1490560...,0.163392


#### CAPRATE BRUTO

In [110]:
# df_venta = pd.read_excel(r"C:\Users\blavin\Documents\Github_myself\portal_inmobiliario_web_scraper\results\metro_chile_españa\results_invesment_venta_departamento_2203_16_07_24.xlsx")

df_venta = pd.read_excel(r"C:\Users\blavin\Documents\Github_myself\portal_inmobiliario_web_scraper\results\Depas_general_arriagada\results_invesment_venta_departamento_1603_17_07_24.xlsx")

In [111]:
df_venta["caprate_bruto"]=np.NaN
for n_dorm in pd.Series(df_venta["n_dormitorios"].unique()).sort_values().values:
    if n_dorm in results_precio_arriendo.keys():
        df_venta.loc[df_venta["n_dormitorios"]==n_dorm, "caprate_bruto" ] = results_precio_arriendo[n_dorm] * 12 / df_venta.query(f"n_dormitorios=={n_dorm}").precio


In [112]:
df_venta

,Unnamed: 0,latitud,longitud,precio,precio_UF,dias_desde_publicacion,n_dormitorios,n_banos,superficie_total,superficie_util,...,bodegas,antiguedad,cantidad_pisos_edificio,piso_unidad,tipo_inmueble,orientacion,titulo,ubicacion,link,caprate_bruto
0,0,-33.557235,-70.585874,127000000,3378.027923,5,2,1,62.00,62.00,...,1,25,7,0,-,-,Departamento En Venta De 2 Dorm. En La Florida,"Vicuña Mackenna 6640, La Florida, La Florida, ...",https://www.portalinmobiliario.com/MLC-1503001...,NaN
1,1,-33.559324,-70.586055,90000000,2393.878056,9,3,2,65.96,61.14,...,0,16,11,4,Departamento,N,Amplio Departamento (23107),"Amplio Departamento, Los Quillayes, La Florida...",https://www.portalinmobiliario.com/MLC-1501231...,0.057600
2,2,-33.557547,-70.586747,105000000,2792.857732,15,3,1,80.00,80.00,...,0,0,4,0,Departamento,O,Departamento Remodelado En La Florida,"La Paloma, La Florida, Chile, Los Quillayes, L...",https://www.portalinmobiliario.com/MLC-2546023...,0.049371
3,3,-33.558409,-70.587695,109028110,2900.000000,120,3,1,60.00,60.00,...,0,0,4,4,Departamento,N,Departamento En Venta En La Florida,"El Colibrí, La Florida, Chile, Los Quillayes, ...",https://www.portalinmobiliario.com/MLC-2313468...,0.047547
4,4,-33.559993,-70.585957,107000000,2846.055022,270,3,1,56.00,56.00,...,0,17,17,3023,NaN,SP,Metro San José De La Estrella / Vicuña Mackenna,"Los Quillayes, La Florida, RM (Metropolitana)",https://www.portalinmobiliario.com/MLC-1473135...,0.048449


In [102]:
df_venta

,Unnamed: 0,titulo,precio,ubicacion,n_dormitorios,n_banos,superficie_total,superficie_util,latitud,longitud,dias_desde_publicacion,estacionamientos,bodegas,cantidad_pisos_edificio,piso_unidad,tipo_casa,orientacion,antiguedad,link,caprate_bruto
0,46,"Departamento En Venta Brown Norte, Plaza Ñuñoa",262803729.0,"Brown Norte 40, Plaza Ñuñoa, Ñuñoa, RM (Metrop...",3,2,9631,9117,-33.454849,-70.592262,60,2,1.0,18.0,5.0,NaN,NO,14.0,https://www.portalinmobiliario.com/MLC-1481967...,0.031963
1,105,Departamento Alonso De Ercilla 3d/2b/e/b Metro...,272578975.0,Departamento Alonso De Ercilla 3d/2b/e/b Metro...,3,2,90,8291,-33.451872,-70.598091,19,1,1.0,7.0,3.0,NaN,P,3.0,https://www.portalinmobiliario.com/MLC-1497951...,0.030817
2,218,Campoamor - Oportunidad / 3d - 3b / Nuevo!!!,269947178.0,"Chile España 451, Ñuñoa, Chile, Plaza Ñuñoa, Ñ...",3,3,8706,7986,-33.450790,-70.597848,60,0,NaN,NaN,NaN,NaN,N,NaN,https://www.portalinmobiliario.com/MLC-2425596...,0.031117
3,160,Vendo. Plaza Ñuñoa. Hermoso Departamento. 2d/2...,270699120.0,"Avenida Irarrázaval 3360, Ñuñoa, Chile, Plaza ...",2,2,8031,7531,-33.454953,-70.595827,120,1,1.0,12.0,10.0,NaN,SP,3.0,https://www.portalinmobiliario.com/MLC-1469578...,0.027484
4,65,Venta Depto 2d2b+e+b Metro Chile España - Ñuñoa,206784050.0,Avda. Irarrázaval Cercano A Metro Chile España...,2,2,7949,7417,-33.454849,-70.592261,300,1,1.0,18.0,0.0,NaN,Nor-Poniente,16.0,https://www.portalinmobiliario.com/MLC-1922169...,0.035980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,129,Venta Departamento En Ñuñoa 2d 2b 1e Inversion...,165427240.0,"Ñuñoa, Chile, Plaza Ñuñoa, Ñuñoa, RM (Metropol...",2,2,54,5,-33.454918,-70.598292,60,1,1.0,21.0,9.0,NaN,SP,1.0,https://www.portalinmobiliario.com/MLC-2420005...,0.044974
246,106,"Excelente Ubicación, Departamento Con Estacion...",150388400.0,"Ángel Pino 3412, Ñuñoa, Chile, Plaza Ñuñoa, Ñu...",1,1,42,3,-33.453524,-70.595648,365,1,1.0,9.0,3.0,NaN,S,11.0,https://www.portalinmobiliario.com/MLC-1492059...,0.036067
247,41,"Ñuñoa, Brown Norte 562, Depto 1d+1e - Mb Propi...",116551010.0,"Brown Norte 562, Plaza Ñuñoa, Ñuñoa, RM (Metro...",1,1,40,0,-33.449930,-70.592090,1,0,NaN,0.0,NaN,NaN,NaN,24.0,https://www.portalinmobiliario.com/MLC-2579766...,0.046538
248,141,"Vendo Como Nuevo, Dpto Lado A Plaza Ñuñoa",219943035.0,"Plaza Ñuñoa, Ñuñoa, RM (Metropolitana)",2,2,81,0,-33.454917,-70.598291,46,1,1.0,0.0,5.0,NaN,NaN,4.0,https://www.portalinmobiliario.com/MLC-1489995...,0.033827
